In [11]:
import tushare as ts
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels import regression
import ffn
from sklearn.svm import SVR
import matplotlib.pyplot as plt

In [12]:
#获取上证50的code
def get_stockID():
    data1 = ts.get_sz50s()
    stocks_ID = data1.loc[:,'code']
    return stocks_ID


In [13]:
#获取股票2017-3月到6月的股票数据作为训练集 将2017-7月份的数据作为测试集
def init(code):
        
    train_data = ts.get_hist_data(code,start='2017-03-01',end='2017-06-30')
    test_data = ts.get_hist_data(code,start='2017-07-01',end='2017-08-31')
           
    return train_data,test_data


In [14]:
#将需要的特征提取出来

def extract_feature(train_data,test_data):
    train_data = train_data.loc[:,['open','high','close','low','price_change','p_change','turnover']]
    test_data = test_data.loc[:,['open','high','close','low','price_change','p_change','turnover']]
    
    return train_data,test_data


In [15]:
#计算开盘到收盘之间的价格均值
def cal_mean(train_data,test_data):
    first_data_open = train_data.loc[:,['open']]
    first_data_close = train_data.loc[:,['close']]

    second_data_open = test_data.loc[:,['open']]
    second_data_close = test_data.loc[:,['close']]
    
    first_data_open.rename(columns = {'open':'mean'},inplace=True)
    first_data_close.rename(columns = {'close':'mean'},inplace=True)
    first_data_mean = (first_data_close + first_data_open)/2
    
    second_data_open.rename(columns = {'open':'mean'},inplace=True)
    second_data_close.rename(columns = {'close':'mean'},inplace=True)
    second_data_mean = (second_data_close + second_data_open)/2
    
    train_data['mean'] = first_data_mean
    test_data['mean'] = second_data_mean
    return train_data,test_data




In [16]:
#计算N期收益
def cal_nReturn(train_data, n):
    #训练集的N期平均收益率
    origin_mean = train_data.loc[:,['mean']]
    origin_mean_n = origin_mean.shift(n)
    train_simpleret = (origin_mean - origin_mean_n) / origin_mean_n
    train_data['n_simpleret'] = train_simpleret
    #填充缺失的值
    train_data = train_data.fillna(method='bfill')
    return train_data
    
    

In [17]:
#训练和预测股票的N期收益
def Train(n):
    #使用scikit-learn导入SVR进行训练和预测
    x_train = []
    y_label = []
    x_test = []
    stocks_ID = get_stockID()
    for code in stocks_ID:
        train_data,test_data = init(code)
        train_data,test_data = extract_feature(train_data,test_data)
        train_data,test_data = cal_mean(train_data,test_data)
        train_data = cal_nReturn(train_data,n)
        svr_rbf = SVR(kernel='rbf',C=1e3,gamma=0.1)
        X = train_data.iloc[:,-9:-2]
        X1 = test_data.iloc[:,-8:-1]
        y = train_data.iloc[:,-1:]
        for i in range(len(X)):
            x_train.append(list(X.iloc[i]))
        for i in range(len(y)):
            y_label.append(list(y.iloc[i]))
        for i in range(len(X1)):    
            x_test.append(list(X1.iloc[i]))
        
    y_rbf = svr_rbf.fit(x_train,y_label).predict(x_test)
    return y_rbf
        
    

In [18]:
ans = Train(15)
ans

/home/lyz/anaconda2/envs/python3/lib/python3.5/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([-0.01226435,  0.00128179,  0.00536256, ...,  0.05089279,
        0.04711263,  0.04966329])

In [19]:
len(ans)

1989